### __Migraciones__

***

In [1578]:
import pandas as pd

Carga del dataset de Migraciones.

In [1579]:
Dataset_Migraciones = 'Migraciones'
Ruta = ''
Migraciones_DB = pd.read_csv(Ruta + Dataset_Migraciones + '.csv')

Comienzo del filtrado. Lo primero es quedarnos con las filas de "TOT" en la columna de 'Migration by Gender Code'. Las filas FEM y MAL, en conjunto y sumadas, dan TOT, por ende sería información repetida e innecesaria para los fines del objetivo del estudio.

In [1580]:
Migrantes = Migraciones_DB[Migraciones_DB['Migration by Gender Code'] == 'TOT']

Eliminamos las filas pertenecientes a "Refugiados", ya que no es un país.

In [1581]:
Migrantes = Migrantes[~(Migrantes['Country Origin Name'] == 'Refugees')]

Eliminamos las columnas que no incluyen a Argentina como país de origen o destino.

In [1582]:
Migrantes = Migrantes[(Migrantes['Country Origin Code'] == 'ARG') | 
                      (Migrantes['Country Dest Code'] == 'ARG')]

Filtramos las columnas a las que nos importan. Además, reseteamos el índice para no generar problemas.

In [1583]:
Columnas = ['Country Origin Code', 'Country Dest Code', 
            '1960 [1960]', '1970 [1970]', '1980 [1980]', '1990 [1990]', '2000 [2000]']

Migrantes = Migrantes[Columnas]
Migrantes = Migrantes.reset_index(drop=True)

Chequeamos que los países de Origen sean los mismos exactamente que los Destino. "Refugees" era el que no coincidía, pero ya fue eliminado.

In [1584]:
Origen = sorted(list(Migrantes['Country Origin Code'].unique()))
Destino = sorted(list(Migrantes['Country Dest Code'].unique()))

print(f"Hay en total {len(Destino)} países distintos.")
print(f"Argentina aparece {list(Migrantes['Country Origin Code']).count('ARG')} veces.")
assert Origen == Destino, "No están los mismos países en las dos columnas."

Hay en total 231 países distintos.
Argentina aparece 231 veces.


Chequeamos si cada país aparece la misma cantidad de veces en cada columna.

In [1585]:
from collections import Counter

Paises_Origen = list(Migrantes['Country Origin Code'])
Paises_Destino = list(Migrantes['Country Dest Code'])

Frecuencias_Origen = Counter(Paises_Origen)
Frecuencias_Destino = Counter(Paises_Destino)

assert Frecuencias_Origen == Frecuencias_Destino, "No aparecen la misma cantidad de veces los países."

Cambiamos los nombres de los atributos a los definitivos.

In [1586]:
Nuevos_Nombres = {'Country Origin Code': 'id_pais_origen',
                  'Country Dest Code': 'id_pais_destino',
                  '1960 [1960]': 'cantidad_en_1960',
                  '1970 [1970]': 'cantidad_en_1970',
                  '1980 [1980]': 'cantidad_en_1980',
                  '1990 [1990]': 'cantidad_en_1990',
                  '2000 [2000]': 'cantidad_en_2000'}

Migrantes = Migrantes.rename(columns=Nuevos_Nombres)

Ya que las columnas de "cantidad_en_x" deben tener enteros, si hay strings, los convertimos en NULLS.

In [1587]:
Columnas_Numericas = ['cantidad_en_1960', 'cantidad_en_1970', 'cantidad_en_1980', 
                      'cantidad_en_1990', 'cantidad_en_2000']

for Columna in Columnas_Numericas:
    Migrantes[Columna] = pd.to_numeric(Migrantes[Columna], errors='coerce')

Verificamos si hay NULLS en las columnas.

In [1588]:
Count_Nulls_By_Columns = Migrantes.isnull().sum()

Llenamos los NULLS con 0. Esto es porque en la tabla había "." para cuando no había habido migraciones en ninguna de las columnas numéricas, por lo que 0 es un relleno correcto.

In [1589]:
for Columna in Columnas_Numericas:
    Migrantes[Columna] = Migrantes[Columna].fillna(0)

Por las dudas, borramos espacios entre palabras (que suponemos no los hay, pero por si los hubiere perdidos por ahí) y al principio y al final.

In [1590]:
Columnas_String = ['id_pais_origen', 'id_pais_destino']

for Columna in Columnas_String:
    Migrantes[Columna] = Migrantes[Columna].str.strip().str.replace(r'\s+', ' ', regex=True)

Migrantes

,id_pais_origen,id_pais_destino,cantidad_en_1960,cantidad_en_1970,cantidad_en_1980,cantidad_en_1990,cantidad_en_2000
0,AFG,ARG,6.0,9.0,13.0,21.0,0.0
1,ALB,ARG,4.0,23.0,45.0,147.0,0.0
2,DZA,ARG,71.0,34.0,0.0,231.0,0.0
3,ASM,ARG,0.0,0.0,0.0,0.0,0.0
4,AND,ARG,0.0,4.0,7.0,1.0,0.0
...,...,...,...,...,...,...,...
456,WLF,ARG,0.0,0.0,0.0,0.0,0.0
457,PSE,ARG,75.0,117.0,166.0,2.0,0.0
458,YEM,ARG,4.0,6.0,9.0,0.0,0.0
459,ZMB,ARG,2.0,1.0,0.0,1.0,0.0


Hay una fila que tiene a ARG de los dos lados: la borramos.

In [1591]:
Migrantes = Migrantes[~((Migrantes['id_pais_origen'] == 'ARG') & 
                        (Migrantes['id_pais_destino'] == 'ARG'))]

Creamos cinco DataFrames, cada uno con un año específico. Le agregamos una columna con el año correspondiente. Después, los filtramos: nos quedamos solo con las filas con valores mayores a 0, es decir, con filas con migrantes. Reseteamos el índice y les cambiamos el nombre. Por último, los concatenamos en el DataFrame final.

In [1592]:
DataFrames_Cantidades = []

for Columna in Columnas_Numericas:
    DataFrame = Migrantes[['id_pais_origen', 'id_pais_destino', Columna]]    # Filtrado de columnas.
    #DataFrame = DataFrame[DataFrame[Columna] > 0]                            # Filtrado de filas.
    DataFrame['anio'] = Columna[-4:len(Columna)+1]                           # Crear columna con el año.    
    DataFrame = DataFrame.reset_index(drop=True)                             # Resetear índices.
    DataFrame = DataFrame.rename(columns={Columna: 'migrantes'})             # Renombrar columna.
    DataFrames_Cantidades.append(DataFrame)                                  # Agregar df a lista de dfs.

Migrantes = pd.concat(DataFrames_Cantidades).reset_index(drop=True)

Migrantes

C:\Users\tomas\AppData\Local\Temp\ipykernel_15188\1421965670.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DataFrame['anio'] = Columna[-4:len(Columna)+1]                           # Crear columna con el año.
C:\Users\tomas\AppData\Local\Temp\ipykernel_15188\1421965670.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DataFrame['anio'] = Columna[-4:len(Columna)+1]                           # Crear columna con el año.
C:\Users\tomas\AppData\Local\Temp\ipykernel_15188\1421965670.py:6: SettingWithCopyWa

,id_pais_origen,id_pais_destino,migrantes,anio
0,AFG,ARG,6.0,1960
1,ALB,ARG,4.0,1960
2,DZA,ARG,71.0,1960
3,ASM,ARG,0.0,1960
4,AND,ARG,0.0,1960
...,...,...,...,...
2295,WLF,ARG,0.0,2000
2296,PSE,ARG,0.0,2000
2297,YEM,ARG,0.0,2000
2298,ZMB,ARG,0.0,2000


Armamos un DataFrame para los migrantes recibidos.

In [1593]:
Inmigrantes = Migrantes[Migrantes['id_pais_destino'] == 'ARG'].copy()
Inmigrantes.loc[:, 'emigrantes'] = 0
Inmigrantes = Inmigrantes[['id_pais_origen', 'migrantes', 'emigrantes', 'anio']]
Inmigrantes = Inmigrantes.rename(columns={'id_pais_origen': 'id_pais', 'migrantes': 'inmigrantes'}).reset_index(drop=True)
Inmigrantes

,id_pais,inmigrantes,emigrantes,anio
0,AFG,6.0,0,1960
1,ALB,4.0,0,1960
2,DZA,71.0,0,1960
3,ASM,0.0,0,1960
4,AND,0.0,0,1960
...,...,...,...,...
1145,WLF,0.0,0,2000
1146,PSE,0.0,0,2000
1147,YEM,0.0,0,2000
1148,ZMB,0.0,0,2000


Armamos otro para los inmigrantes.

In [1594]:
Emigrantes = Migrantes[Migrantes['id_pais_origen'] == 'ARG'].copy()
Emigrantes.loc[:, 'inmigrantes'] = 0
Emigrantes = Emigrantes[['id_pais_destino', 'inmigrantes', 'migrantes', 'anio']]
Emigrantes = Emigrantes.rename(columns={'id_pais_destino': 'id_pais', 'migrantes': 'emigrantes'}).reset_index(drop=True)
Emigrantes

,id_pais,inmigrantes,emigrantes,anio
0,AFG,0,0.0,1960
1,ALB,0,0.0,1960
2,DZA,0,3.0,1960
3,ASM,0,0.0,1960
4,AND,0,4.0,1960
...,...,...,...,...
1145,WLF,0,0.0,2000
1146,PSE,0,175.0,2000
1147,YEM,0,68.0,2000
1148,ZMB,0,0.0,2000


Combinamos los datos.

In [1595]:
Migrantes = pd.merge(
    Inmigrantes[['id_pais', 'anio', 'inmigrantes']],
    Emigrantes[['id_pais', 'anio', 'emigrantes']],
    on=['id_pais', 'anio'], 
    how='left'
).reset_index(drop=True)

Migrantes.fillna({'inmigrantes': 0}, inplace=True)
Migrantes.fillna({'emigrantes': 0}, inplace=True)

Última verificación de que no hay valores duplicados.

In [1596]:
Duplicados = Migrantes[Migrantes.duplicated(subset=['id_pais', 'anio'], keep=False)]

Duplicados

,id_pais,anio,inmigrantes,emigrantes


Guardamos el archivo.

In [1597]:
Migrantes.to_csv('Tablas/TABLA. Migrantes.csv', index=False)

Migrantes

,id_pais,anio,inmigrantes,emigrantes
0,AFG,1960,6.0,0.0
1,ALB,1960,4.0,0.0
2,DZA,1960,71.0,3.0
3,ASM,1960,0.0,0.0
4,AND,1960,0.0,4.0
...,...,...,...,...
1145,WLF,2000,0.0,0.0
1146,PSE,2000,0.0,175.0
1147,YEM,2000,0.0,68.0
1148,ZMB,2000,0.0,0.0


### __Regiones__

***

Las regiones se extraen del dataset "Sedes".

In [1598]:
Dataset_Sedes = 'Sedes'
Sedes_DB = pd.read_csv(Ruta + Dataset_Sedes + '.csv')

Arreglamos discordancias de datos entre los códigos de países en Sedes y Migraciones.

In [1599]:
Sedes_DB['pais_iso_3'] = Sedes_DB['pais_iso_3'].replace('GRB', 'GBR')

Nos quedamos con las columna que nos interesa: region_geografica.

In [1600]:
Regiones = pd.DataFrame(Sedes_DB['region_geografica'])

Eliminamos espacios adicionales.

In [1601]:
Regiones['region_geografica'] = Regiones['region_geografica'].str.strip().str.replace(r'\s+', ' ', regex=True)

Nos quedamos con los valores únicos.

In [1602]:
Regiones = Regiones.drop_duplicates()

Las ordenamos alfabéticamente y reseteamos el índice para no generar problemas.

In [1603]:
Regiones = Regiones.sort_values(by='region_geografica').reset_index(drop=True)

Generamos una nueva columna: id, con números identificatorios.

In [1604]:
Regiones['id'] = [x for x in range(1,len(Regiones)+1)]

Cambiamos los nombres de los atributos y ordenamos las columnas

In [1605]:
Regiones = Regiones.rename(columns = {'region_geografica': 'nombre'})
Regiones = Regiones[['id', 'nombre']]

Chequeo final de nulos en la columna "id". Los eliminamos.

In [1606]:
Regiones = Regiones.dropna(subset='id')

Guardamos el documento en un .csv.

In [1607]:
Regiones.to_csv('Tablas/TABLA. Regiones.csv', index=False)

Regiones

,id,nombre
0,1,AMÉRICA CENTRAL Y CARIBE
1,2,AMÉRICA DEL NORTE
2,3,AMÉRICA DEL SUR
3,4,ASIA
4,5,EUROPA CENTRAL Y ORIENTAL
5,6,EUROPA OCCIDENTAL
6,7,OCEANÍA
7,8,ÁFRICA DEL NORTE Y CERCANO ORIENTE
8,9,ÁFRICA SUBSAHARIANA


### __Países__

***

Usamos los códigos y nombres de la base de datos "Migraciones.csv", ya que son más exhaustivos que los de las otras tablas. El procedimiento para encontrar los valores únicos es:

1. Armar tres DataFrames: uno con los países de origen -código y nombre-, otro con los países de destino -código y nombre- y otro con los países de "Sedes". De esa forma obtenemos todos los países que están en una u otra columna pero no en las otras.
2. Concatenarlos verticalmente, llamando las columnas "id" y "nombre". Este DataFrame se llama "Países".
3. Se eliminan las filas con valores duplicados en la columna "id". 

In [1608]:
Paises_Origen = Migraciones_DB[['Country Origin Code', 'Country Origin Name']]
Paises_Destino = Migraciones_DB[['Country Dest Code', 'Country Dest Name']]
Paises_Sedes = Sedes_DB[['pais_iso_3', 'pais_castellano']]

Renombramos las columnas para que se llamen igual.

In [1609]:
Nuevos_Nombres_Origen = {'Country Origin Code': 'id',
                         'Country Origin Name': 'nombre'}

Nuevos_Nombres_Destino = {'Country Dest Code': 'id',
                          'Country Dest Name': 'nombre'}

Nuevos_Nombres_Sedes = {'pais_iso_3': 'id',
                        'pais_castellano': 'nombre'}

Paises_Origen = Paises_Origen.rename(columns = Nuevos_Nombres_Origen)
Paises_Destino = Paises_Destino.rename(columns = Nuevos_Nombres_Destino)
Paises_Sedes = Paises_Sedes.rename(columns = Nuevos_Nombres_Sedes)

Concatenamos verticalmente los dos DataFrames y creamos "Paises".

In [1610]:
Paises = pd.concat([Paises_Origen, Paises_Destino, Paises_Sedes], axis=0)

Eliminamos las filas pertenecientes a "Refugiados", ya que no es un país.

In [1611]:
Paises = Paises[~(Paises['id'] == 'zzz')]

Eliminamos espacios adicionales.

In [1612]:
for Columna in Paises.columns:
    Paises[Columna] = Paises[Columna].str.strip().str.replace(r'\s+', ' ', regex=True)

Filtramos los valores repetidos de las columna "id". Además, reseteamos el índice para no generar problemas.

In [1613]:
Paises = Paises.drop_duplicates(subset=['id']).reset_index(drop=True)

Eliminamos los duplicados en la columna "pais_iso_3" en el DataFrame "Sedes". Esto es para que no se generen filas de más en el join.

In [1614]:
Paises_En_Sedes = Sedes_DB.drop_duplicates(subset='pais_iso_3')

Además, eliminamos espacios adicionales.

In [1615]:
for Columna in ['region_geografica', 'pais_iso_3']:
    Paises_En_Sedes[Columna] = Paises_En_Sedes[Columna].str.strip().str.replace(r'\s+', ' ', regex=True)

C:\Users\tomas\AppData\Local\Temp\ipykernel_15188\898895942.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Paises_En_Sedes[Columna] = Paises_En_Sedes[Columna].str.strip().str.replace(r'\s+', ' ', regex=True)


Agregamos el 'id_region' correspondiente a cada país. Este se extrae del correspondiente al país en el dataset "Sedes". Si no se encuentra, queda NULL.

In [1616]:
Paises = pd.merge(Paises, Paises_En_Sedes, left_on='id', right_on='pais_iso_3', how='left')[['id', 'nombre', 'region_geografica']]

Hacemos un LEFT_JOIN con el DataFrame "Regiones" para extraer los id de las regiones correspondientes.

In [1617]:
Paises = pd.merge(Paises, Regiones, left_on='region_geografica', right_on='nombre', how='left')

Filtramos las columnas a las que necesitamos y cambiamos nombres.

In [1618]:
Paises = Paises[['id_x', 'nombre_x', 'id_y']]

Paises = Paises.rename(columns={'id_x': 'id', 'nombre_x': 'nombre', 'id_y': 'id_region'})

Contamos los NULLS y los reemplazamos con "-" (esto es opcional).

NOTA: Este punto es problemático porque hay demasiados NULLS.

In [1619]:
Paises['id_region'] = Paises['id_region'].fillna('-')

Ordenamos alfabéticamente por id.

In [1620]:
Paises = Paises.sort_values(by='id').reset_index(drop=True)

Chequeo final de nulos en la columna "id". Los eliminamos.

In [1621]:
Paises = Paises.dropna(subset='id')

Chequeo final de que no hay valores duplicados en la clave.

In [1622]:
Duplicados = Paises[Paises.duplicated(subset=['id'], keep=False)]

Duplicados

,id,nombre,id_region


Guardamos el documento.

In [1623]:
Paises.to_csv('Tablas/TABLA. Paises.csv', index=False)

Paises

,id,nombre,id_region
0,ABW,Aruba,-
1,AFG,Afghanistan,-
2,AGO,Angola,9.0
3,AIA,Anguilla,-
4,ALB,Albania,-
...,...,...,...
227,XKX,Kosovo,-
228,YEM,"Yemen, Rep.",-
229,ZAF,South Africa,9.0
230,ZMB,Zambia,-


### __Sedes__

***

Se extraen del dataset "Sedes". Nos quedamos con las columnas que nos interesan y conforman los atributos.

In [1624]:
Sedes = Sedes_DB[['sede_id', 'sede_desc_castellano', 'pais_iso_3']]

Cambiamos los nombres de las columnas.

In [1625]:
Sedes = Sedes.rename(columns={'sede_id': 'id', 'pais_iso_3': 'id_pais', 'sede_desc_castellano': 'nombre'})

Borramos espacios adicionales.

In [1626]:
for Columna in Sedes.columns:
    Sedes[Columna] = Sedes[Columna].str.strip().str.replace(r'\s+', ' ', regex=True)

Ordenamos alfabéticamente por id.

In [1627]:
Sedes = Sedes.sort_values(by='id').reset_index(drop=True)

Chequeo final de nulos en la columna "id". Los eliminamos.

In [1628]:
Sedes = Sedes.dropna(subset='id')

Chequeo final de que no hay valores duplicados en la clave.

In [1629]:
Duplicados = Sedes[Sedes.duplicated(subset=['id', 'nombre'], keep=False)]

Duplicados

,id,nombre,id_pais


Guardamos el documento.

In [1630]:
Sedes.to_csv('Tablas/TABLA. Sedes.csv', index=False)

Sedes

,id,nombre,id_pais
0,CALEG,Consulado General en Porto Alegre,BRA
1,CANTO,Consulado en Antofagasta,CHL
2,CAREN,Consulado General en Punta Arenas,CHL
3,CASUN,Consulado General en Asunción,PRY
4,CATLA,Consulado General en Atlanta,USA
...,...,...,...
159,EURUG,Embajada en Uruguay,URY
160,EVENE,Embajada en Venezuela,VEN
161,EVIET,Embajada en Vietnam,VNM
162,OCCAT,Oficina Comercial y Cultural Argentina en Taiwán,CHN


### __Secciones__

***

Para armar esta tabla utilizamos el dataset "Secciones.csv".

In [1631]:
Dataset_Secciones = 'Secciones'
Secciones_DB = pd.read_csv(Ruta + Dataset_Secciones + '.csv')

Quitamos las columnas que no sirven y cambiamos los nombres de los atributos.

In [1632]:
Secciones = Secciones_DB[['sede_id', 'sede_desc_castellano']]
Secciones = Secciones.rename(columns={'sede_id': 'id_sede', 'sede_desc_castellano': 'nombre'})

Borramos espacios adicionales.

In [1633]:
for Columna in Secciones.columns:
    Secciones[Columna] = Secciones[Columna].str.strip().str.replace(r'\s+', ' ', regex=True)

Ordenamos alfabéticamente por id.

In [1634]:
Secciones = Secciones.sort_values(by='id_sede').reset_index(drop=True)

Chequeo final de nulos en la columna "id". Los eliminamos.

In [1635]:
Secciones = Secciones.dropna(subset='id_sede')

Chequeo de si no hay valores duplicados en la clave.

In [1636]:
Duplicados = Secciones[Secciones.duplicated(subset=['id_sede', 'nombre'], keep=False)]

Eliminar los duplicados.

In [1637]:
Secciones = Secciones.drop_duplicates(subset=['id_sede', 'nombre'], keep=False)

Guardamos el documento.

In [1638]:
Secciones.to_csv('Tablas/TABLA. Secciones.csv', index=False)

Secciones

,id_sede,nombre
0,CBARC,"Sección Cultura, Educación y Redes Sociales"
1,CBARC,Sección Administrativa
2,CBARC,Sección Consular (Asistencia al ciudadano/Visas)
3,CBARC,Sección Consular (Documentación/Asuntos previs...
4,CBARC,"Sección Comercial, Turismo e Inversiones"
...,...,...
510,EVENE,Sección Política
511,EVIET,Sección Prensa y Cultura
512,EVIET,Sección Política y Cooperación
513,EVIET,"Sección Económica, Comercial y de Inversiones"
